# Campaign optimization.

Se plantea un problema de inversión en *paid marketing* entre los distintos canales y días de la semana.  
Rentabilizar al máximo la inversiones en campañas de marketing maximizando el ROAS (Return Of Assets), $\ ROAS = \frac{Ingresos}{Inversión}$.  
Donde el revenue (ingresos) los podemos considerar como el tráfico x Ticket Medio.$\ revenue = \frac{CR}{CPC}·Avg.Order Value$.

Los canales posibles para las campañas son 6: Instagram Stories, Facebook, Google Shopping, Google Adwords, Red display e Influencers.

Los datos de los que disponemos hasta la fecha son:  

  Canal             |  CPC  |   CR  |
  :----------------:|:-----:|:-----:|
  Facebook          | 0.25€ | 0.015 |
  Instagram Stories | 0.35€ | 0.022 |
  Google Shopping   | 0.20€ | 0.010 |
  Google Adwords    | 0.17€ | 0.008 |
  Red display       | 0.05€ | 0.001 |


Por otro lado, existe una variación respecto a los valores anteriores, que son una media, dependiendo del día de la semana.

  Canal             | Lunes | Martes | Miércoles | Jueves | Viernes| Sábado | Domingo |
  :----------------:|:-----:|:------:|:---------:|:------:|:------:|:------:|:-------:|
  Facebook          |   8% |      8% |        3% |     -- |  0.97% |  0.92% |   0.92% |
  Instagram Stories |   3% |      2% |        -- |     -- |  0.95% |  0.88% |   0.86% |
  Google Shopping   |  10% |      4% |       16% |     -- |  0.96% |  0.90% |   0.88% |
  Google Adwords    |   6% |     13% |       17% |     -- |  1.04% |  0.94% |   0.94% |
  Red display       |  50% |     50% |       25% |     -- |     -- |     -- |      -- |


También hay que tener en cuenta que el presupuesto total para las campañas es 10k €.  
Otras decisiones vienen impuestas por la estrategia de Marketing de la empresa como, por ejemplo, hay que estar presente todos los días, no podemos desaparecer de ningún canal, por tanto, una presencia mínima del 1% del presupuesto total para todos los canales y días.
Por otro lado, ningún día puede consumir más del 20% del prespuesto.  
Además para no depender de las variaciones de precio de un único canal, el presupuesto máximo en cada canal no puede pasar del 25%.  

### Planteamiento  
  
<li>Variables de decisión: Inversión diaria por canal. Por tanto, 5 canales x 7 días = 35 variables.</li>
<li>Función Ojetivo: Maximizar el ROAS</li>
<li>Restricción 1: "*También hay que tener en cuenta que el presupuesto total para las campañas es 10k €.*"</li>
<li>Restricción 2: "*... una presencia mínima del 1% del presupuesto total para todos los canales y días*"</li>
<li>Restricción 3: "*Por otro lado, ningún día puede consumir más del 20% del prespuesto.*"</li>
<li>Restricción 4: "*..., el presupuesto máximo en cada canal no puede pasar del 25%.*"</li>  



## Formulación General

Decision Variables:

\begin{align*}
x_{IG,lunes}, x_{IG,martes}, x_{IG,miércoles}, & \quad ... & x_{FB,lunes}, x_{FB,martes}, & \quad ... & \quad x_{i,j} & \quad ... & \quad x_{canal, día}\\
\end{align*}

  
Objetive Function:

\begin{align*}
\underset{x_{ij}}{\max} & \quad \sum_{i=1}^{n}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{ij}}{CPC_{i}} x_{ij} \frac{AOV}{B} \quad \to \quad \underset{x_{ij}}{\max} \sum_{i=1}^{n}\sum_{j=1}^{m} C_{ij}x_{ij} \quad \text{siendo} \quad C_{ij} =  \sum_{i=1}^{n}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{i, j}}{CPC_{i}}\frac{AOV}{B}
\end{align*}
  
  
  
Constraints:

\begin{align*}
\text{s.t.:} \quad  &\\
\text{(1)} \quad   &\sum_{i=1}^{n}\sum_{j=1}^{m} x_{ij} \leq B \quad \quad \quad \text{Constraint: Max. budget for the campaign}\\
\text{(2)} \quad   & x_{ij} \geq k_{1} · B \quad \forall i,j \quad \quad \quad \text{Constraint: Daily minimum budget for each channel constraint}\\
\text{(3)} \quad   &\sum_{i=1}^{n} x_{ij} \leq k_{2} · B \quad \forall j \quad \quad \text{Daily maximum budget constraint}\\
\text{(4)} \quad   &\sum_{j=1}^{m} x_{ij} \leq k_{3} · B \quad \forall i \quad \quad \text{Constraint: Maximum budget for any channel constraint}\\
\end{align*}

### Implementación del modelo en Pyomo como \AbstractModel()". Obtención de la sensibilidad asociada a cada una de las restricciones y la interpretación de sus valores.

In [1]:
%%writefile paid_marketing_primal.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

# create indexes
model.n_channel = Param(within=NonNegativeIntegers) # number of Channel channels.
model.n_day = Param(within=NonNegativeIntegers) # number of days. In this case, weekdays.

model.Channel = RangeSet(1,model.n_channel) # Channel channels.
model.Day = RangeSet(1,model.n_day) # weekdays.


# create parametres
model.CPC = Param(model.Channel, within=NonNegativeReals) # CPC per Channel channel.
model.CR = Param(model.Channel, within=NonNegativeReals) # CR per Channel channel.
model.daily_variation = Param(model.Channel, model.Day, within=NonNegativeReals) # variation per weekday and channel
model.B = Param(within=NonNegativeReals) # max. budget
model.AOV = Param(within=NonNegativeReals) # average order value

model.K1 = Param(within=NonNegativeReals) # K1, percentage of budget. Minimum budget percentage for each channel and day (PERCENTAGE_MIN).
model.K2 = Param(within=NonNegativeReals) # K2, percentage of budget. Maximum budget percentage to be complied daily (PERCENTAGE_EVERYDAY_MAX).
model.K3 = Param(within=NonNegativeReals) # K3, percentage of budget. Maximum budget percentage to be complied for all channel (PERCENTAGE_ALL_CHANNEL_MAX).

# create variables
model.x = Var(model.Channel, model.Day, domain=NonNegativeReals) # investment per Channel channel.


# create Objective Function
def objective_function_expression(model):
    return sum(sum(model.CR[i]/model.CPC[i]*model.daily_variation[i,j]*model.x[i,j] 
                   for i in model.Channel) for j in model.Day) * model.AOV / model.B
model.OF = Objective(rule=objective_function_expression, sense = maximize)


# create constraints
# (1) total budget constratints
def total_budget_constraint_rule(model):
     return sum(sum(model.x[channel,day] for channel in model.Channel) for day in model.Day) <= model.B
model.Total_Budget_Constraint = Constraint(rule = total_budget_constraint_rule)

# (2) Min. budget for each day constraint, K1.
def min_budget_constraint_rule(model, channel, day):
    return model.x[channel,day] >= model.K1 * model.B
model.Min_Budget_Constraint = Constraint(model.Channel, model.Day, rule = min_budget_constraint_rule)

# (3) Max. budget for each day constraint, K2.
def max_budget_everyday_constraint_rule(model, day):
    return sum(model.x[channel,day] for channel in model.Channel) <= model.K2 * model.B
model.Max_Budget_Everyday_Constraint = Constraint(model.Day, rule = max_budget_everyday_constraint_rule)

# (4) Max. budget for any channel constraint, K3.
def max_budget_channel_constraint_rule(model, channel):
    return sum(model.x[channel,day] for day in model.Day) <= model.K3 * model.B
model.Max_Budget_Channel_Constraint = Constraint(model.Channel, rule = max_budget_channel_constraint_rule)



Overwriting paid_marketing_primal.py


In [2]:
%%writefile paid_marketing_primal.dat

# one way to input the data in AMPL format 
# for indexed parameters, the indexes are given before the value

param n_channel := 5; # unique channels in Channel channel 
param n_day := 7; # number of weekdays
param B := 10000 ; # max. budget (B)
param AOV := 40; # avg. order value (AOV)
param K1 := 0.01 ; # K1, percentage of budget. Minimum budget percentage for each channel and day (PERCENTAGE_MIN).
param K2 := 0.20 ; # K2, percentage of budget. Maximum budget percentage to be complied daily (PERCENTAGE_EVERYDAY_MAX).
param K3 := 0.25 ; # K3, percentage of budget. Maximum budget percentage to be complied for all channel (PERCENTAGE_ALL_CHANNEL_MAX).

param CPC :=
1	0.25
2	0.35
3	0.20
4	0.17
5	0.05
;

param CR:=
1	0.015
2	0.022
3	0.010
4	0.008
5	0.001
;

param daily_variation: 1 2 3 4 5 6 7 :=
1	1.08 	1.08 	1.03 	1.00 	0.97 	0.92 	0.92 
2	1.03 	1.02 	1.00 	1.00 	0.95 	0.88 	0.86 
3	1.10 	1.04 	1.16 	1.00 	0.96 	0.90 	0.88 
4	1.06 	1.13 	1.17 	1.00 	1.04 	0.94 	0.94 
5	1.50 	1.50 	1.25 	1.00 	1.00 	1.00 	1.00 
;



Overwriting paid_marketing_primal.dat


In [3]:
!pyomo solve paid_marketing_primal.py paid_marketing_primal.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.01] Creating model
[    0.05] Applying solver
[    0.10] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: feasible
      Function Value: 2.2421983193277306
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    x : Size=35, Index=x_index
        Key    : Lower : Value  : Upper : Fixed : Stale : Domain
        (1, 1) :     0 : 1000.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 : 1000.0 :  None : False : False : NonNegativeReals
        (1, 3) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 4) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 5) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 6) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 7) :     0 :  100.0 :  None : False : False : NonNegativ

In [4]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2.24219831932773
  Upper bound: 2.24219831932773
  Number of objectives: 1
  Number of constraints: 49
  Number of variables: 36
  Number of nonzeros: 141
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.027352571487426758
# ----------------------------------------------------------
#   Solution Information
# ----------

La optimización del modelo arroja una solución única al ser el *GAP = 0*.  
Se obitiene un ROAS del 2,24. Lo que quiere decir que por cada euro invertido obtenemos 2,24 euros de ingresos.  
Para conseguir dicho ROAS, la inversión (€) por canal y día de la siguiente manera:  

  Canal             | Lunes | Martes | Miércoles | Jueves | Viernes| Sábado | Domingo |
  :----------------:|:-----:|:------:|:---------:|:------:|:------:|:------:|:-------:|
  Facebook          | 1000  | 1000 |  100 | 100  | 100  | 100  | 100 |
  Instagram Stories |  400  | 100  | 100  | 1600 | 100  | 100  | 100 |
  Google Shopping   |  400  | 100  | 1600 | 100  | 100  | 100  | 100 |
  Google Adwords    |  100  | 700  | 100  | 100  | 600  | 100  | 100 |
  Red display       |  100  | 100  | 100  | 100  | 100  | 100  | 100 |
  
Cumpliendo las condiciones de:
<li> Presupuesto por canal mayor o igual al 1% del presupuesto total </li>

Como era de esperar las RRSS son los canales que más convierten y en los que la relación CPA inversión es mejor.  


La inversión por canal es:

  Canal             | Inversión | 
  :----------------:|:-----:|
  Facebook          | 2.500 |
  Instagram Stories | 2.500 |
  Google Shopping   | 2.500 |
  Google Adwords    | 1.800 |
  Red display       |   700 |


Cumpliendo las condiciones de:
<li> Presupuesto máximo 10k € </li>
<li> En ningún canal se invierte más del 25% del presupuesto total </li>


Y la inversión por día es:

  |          | Lunes | Martes | Miércoles | Jueves | Viernes| Sábado | Domingo |
  |:--------:|:-----:|:------:|:---------:|:------:|:------:|:------:|:-------:|
  |Inversión | 2.000 |  2.000 |     2.000 |  2.000 |  1.000 |    500 |     500 |

En los que se cumple que:
<li> Presupuesto máximo 10k € </li>
<li> Ningún día supera el 20% del presupuesto total</li>


La **sensibilidad** se obtiene que la solución dual.
Destacamos que si, de manera independiente,  aumentáramos en una unidad el presupuesto Máximo diario para el jueves aumentaría la Función Objetivo, es decir, el ROAS en 9 puntos.  
o que si aumentáramos el presupuesto máximo para el canal Shopping aumentaría el ROAS en 7 puntos.
Si miráramos los presupuestos mínimos, al reducir el mínimo de Display del lunes y martes aumentaría la FO en 9 puntos ya que reduciríamos la inversión en ese canal esos dos que no nos dan buen retorno.



### c) (1 puntos) Formula el problema dual del modelo definido en a). Impleméntalo en Pyomo y comprueba que se cumple el Teorema de Dualidad Fuerte así como la equivalencia con las sensibilidades calculadas en b).

Definimos el *dual problem* y comprobamos que los resultados son los equivalentes al *primal*.  

Resolvemos: $A^T·y_{i,j} = c_{i,j}$


## Formulación General

DUAL VARIABLES: 


<li>$y_{1}$ for Constraint 1</li> 
<li>$y_{2·i,j}$ for Constraint 2</li>
<li>$y_{3·j}$ for Constraint 3</li>
<li>$y_{4·i}$ for Constraint 4</li>  
    
OBJECTIVE FUNCTION:

\begin{align*}
\underset{y_{i,j}}{\min} & \quad -y_{1}B + \sum_{i=1}^{n}\sum_{j=1}^{m}y_{2·i,j}K_{1}B - \sum_{j=1}^{m}y_{3·j}K_{2}B - \sum_{i=1}^{n}y_{4·i}K_{3}B\\
\end{align*}
  
  
  
CONSTRAINTS:

\begin{align*}
\text{s.t.:}\\
&\text{(1)} \quad   -y_{1} + y_{2·i,j} - y_{3·j} - y_{4·i} \leq C_{ij} \quad , \quad C_{ij} =  \sum_{i=1}^{n}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{i, j}}{CPC_{i}}\frac{AOV}{B}\\
\end{align*}

In [5]:
%%writefile paid_marketing_dual.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

# create indexes
model.n_channel = Param(within=NonNegativeIntegers) # number of Channel channels.
model.n_day = Param(within=NonNegativeIntegers) # number of days. In this case, weekdays.

model.Channel = RangeSet(1,model.n_channel) # Channel channels.
model.Day = RangeSet(1,model.n_day) # weekdays.


# create parametres
model.CPC = Param(model.Channel, within=NonNegativeReals) # CPC per Channel channel.
model.CR = Param(model.Channel, within=NonNegativeReals) # CR per Channel channel.
model.daily_variation = Param(model.Channel, model.Day, within=NonNegativeReals) # variation per weekday and channel
model.B = Param(within=NonNegativeReals) # max. budget
model.AOV = Param(within=NonNegativeReals) # Average Order Value

model.K1 = Param(within=NonNegativeReals) # K1, percentage of budget. Minimum budget percentage for each channel and day (PERCENTAGE_MIN).
model.K2 = Param(within=NonNegativeReals) # K2, percentage of budget. Maximum budget percentage to be complied daily (PERCENTAGE_EVERYDAY_MAX).
model.K3 = Param(within=NonNegativeReals) # K3, percentage of budget. Maximum budget percentage to be complied for all channel (PERCENTAGE_ALL_CHANNEL_MAX).

# create variables
model.y1 = Var(domain=NonNegativeReals) # Max. budget for the whole paid marketing.
model.y2 = Var(model.Channel, model.Day, domain=NonNegativeReals) # Min. budget for each day constraint.
model.y3 = Var(model.Day, domain=NonNegativeReals) # Max. budget for each day constraint.
model.y4 = Var(model.Channel, domain=NonNegativeReals) # Min. budget for any channel constraint.

# Objective Function Definition
def objective_function_expression(model):
    return -model.B*model.y1 \
            +model.K1*model.B*sum(sum(model.y2[channel, day] for channel in model.Channel) for day in model.Day) \
            -model.K2*model.B*sum(model.y3[day] for day in model.Day ) \
            -model.K3*model.B*sum(model.y4[channel] for channel in model.Channel)
model.Objective_Function = Objective(rule=objective_function_expression,sense=minimize)


# dual constraint
def dual_constraint_rule(model, channel, day):
     return -model.y1 +model.y2[channel,day] -model.y3[day] +model.y4[channel] <= \
        model.CR[channel]/model.CPC[channel],model.daily_variation[channel,day]* model.AOV / model.B
model.Dual_Constraint = Constraint(model.Channel,model.Day, rule=dual_constraint_rule)



Overwriting paid_marketing_dual.py


In [6]:
!pyomo solve paid_marketing_dual.py paid_marketing_primal.dat --solver=glpk --summary

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.03] Creating model
[    0.04] Applying solver
[    0.05] Pyomo Finished
ERROR: Unexpected exception while running model:
        Unexpected expression (type _InequalityExpression):  - y1 + y2[1,1] -
        y3[1] + y4[1]  <=  0.06


No he sabido resolver este error:  
" *ERROR: Unexpected exception while running model:  
        Unexpected expression (type _InequalityExpression):  - y1 + y2[1,1] - y3[1] + y4[1]  <=  0.06* "

In [7]:
!type results.yml

### d) (1.5 punto) Modifica el problema de optimización en a) para imponer alguna restricción lógica o condicional que necesite la incorporación de nuevas restricciones con variables binarias. Resuelve este problema en Pyomo e interpreta los resultados.

El negocio cambia la estrategia de diversificación y le quiere probar concentrar todos sus esfuerzos en un canal, bien Facebook o Google.  

Para realizar esta distinción usaremos una variable binaria, u, que determine si se invierte en los canales de Facebook (Facebook e Instagram) o en los de Google (Shooping, Search y Display).   
Además, asignaremos la variable $y_{i,j}$ a los canales de Facebook y $z_{i,j}$ a los de Google.  
*NOTA*: Esto último obliga a descomponer todos los datos de los canales en 2.  

Además como existe la posibilidad de que solo haya dos canales, que serían los de Facebook, tenemos que subir el porcentaje para el presupuesto máximo por canal al 60% ya que antes era inferior a 50% y no habría solución posible.  

## Formulación General

Variables de decisión:

\begin{align*}
y_{canal,día} \in Facebook channels \\
z_{canal,día} \in Google channels \\
u \in \{0,1\}
\end{align*}

  
Función Objetivo:

\begin{align*}
\underset{x_{ij}}{\max} & \quad \sum_{i=1}^{n}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{ij}}{CPC_{i}}  u y_{ij}   \frac{AOV}{B} + \sum_{i=1}^{n}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{ij}}{CPC_{i}}   (1-u) z_{ij}   \frac{AOV}{B}\\
\end{align*}  
Expresado de forma más sencilla:
\begin{align*}
\underset{x_{ij}}{\max} \sum_{i=1}^{n}\sum_{j=1}^{m} (C_{ij} uy_{ij} + D_{ij} (1-u) z_{j}) \quad \to \quad C_{ij} =  \sum_{i=1}^{FB}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{i, j}}{CPC_{i}}\frac{AOV}{B} , \quad D_{ij} = \sum_{i=1}^{GG}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{i, j}}{CPC_{i}}\frac{AOV}{B}\\
\end{align*}

Constraints:

\begin{align*}
\text{s.t.:} \quad  &\\
\text{(1)} \quad   &\sum_{i=1}^{FB}\sum_{j=1}^{m} u y_{ij} + \sum_{i=1}^{GG}\sum_{j=1}^{m} (1-u)z_{ij} \leq B \quad \quad \quad \text{Constraint: Max. budget for the campaign}\\
\text{(2)} \quad   & u y_{ij} \geq k_{1} · B \quad \forall i,j \quad \quad \quad \quad \quad \text{Constraint: Daily minimum budget for Facebook channels constraint}\\
\text{(3)} \quad   & (1-u) z_{ij} \geq k_{1} · B \quad \forall i,j  \quad \quad  \quad \quad \quad \text{Constraint: Daily minimum budget for Google channels constraint}\\
\text{(4)} \quad   &\sum_{i=1}^{FB} u y_{ij} + \sum_{i=1}^{GG} (1-u) z_{ij} \leq k_{2} · B \quad \forall j \quad \quad \text{Daily maximum budget constraint}\\
\text{(5)} \quad   &\sum_{j=1}^{m} u y_{ij} + \sum_{j=1}^{m} (1-u) z_{ij} \leq k_{3} · B \quad \forall i \quad \quad \text{Constraint: Maximum budget for any channel constraint}\\
\end{align*}



Esta función objetivo es cuadrática en los términos, `u·y` y `u·z`, por tanto, no se puede resolver con linear-programming.  
Para poder resolverla con programación lineal, aplicamos una transformación del tipo:  

  
$FO: \quad z = u·x \quad   u \in \{0,1\} \quad x \in \{x_{min}, x_{max}\}   \quad \to \quad FO: z = x - r$ 


Añadimos dos restricciones:  

<li> $u·x_{min} \quad \leq \quad z \quad \leq \quad u·x_{max}$ </li>
<li> $(1-u)·x_{min} \quad \leq \quad r \quad \leq \quad (1-u)·x_{max}$ </li>  


Solución:  
<li> $u = 1 \to z  =  x$</li>
<li> $u = 0 \to z  =  0$</li>   


## Transformación lineal
Nos generamos una variable auxiliar `r` e introducimos las nuevas restricciones (6), (7), (8) y (9).  
<li> $u·y_{i,j} = y_{i,j}-r$</li>
<li> $u·z_{i,j} = z_{i,j}-r$</li>  

#### Formulación General

\begin{align*}
\underset{x_{ij}}{\max} \sum_{i=1}^{n}\sum_{j=1}^{m} (C_{ij} (y_{ij}-r) + D_{ij} (z_{ij}+r)) \quad \to \quad C_{ij} =  \sum_{i=1}^{FB}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{i, j}}{CPC_{i}}\frac{AOV}{B} , \quad D_{ij} = \sum_{i=1}^{GG}\sum_{j=1}^{m} \frac{CR_{i}\Delta_{i, j}}{CPC_{i}}\frac{AOV}{B}\\
\end{align*}

Constraints:

\begin{align*}
\text{s.t.:} \quad  &\\
\text{(1)} \quad   &\sum_{i=1}^{FB}\sum_{j=1}^{m} (y_{ij}-r) + \sum_{i=1}^{GG}\sum_{j=1}^{m} (z_{ij}+r) \leq B \quad \quad \text{Constraint: Max. budget for the campaign}\\
\text{(2)} \quad   & (y_{ij}-r) \geq k_{1} · B · u \quad \forall i,j \quad \quad \quad \quad \quad \quad \text{Constraint: Daily minimum budget for Facebook channels constraint}\\
\text{(3)} \quad   & (z_{ij}+r) \geq k_{1} · B · (1-u) \quad \forall i,j  \quad  \quad \quad \quad \text{Constraint: Daily minimum budget for Google channels constraint}\\
\text{(4)} \quad   &\sum_{i=1}^{FB} (y_{ij}-r) + \sum_{i=1}^{GG} (z_{ij}+r) \leq k_{2} · B \quad \forall j \quad \quad \text{Daily maximum budget constraint}\\
\text{(5)} \quad   &\sum_{j=1}^{m} (y_{ij}-r) + \sum_{j=1}^{m} (z_{ij}+r) \leq k_{3} · B \quad \forall i \quad \quad \text{Constraint: Maximum budget for any channel constraint}\\
\text{(6)} \quad   &u·y_{min}  \leq  (y_{i,j}-r)  \leq  u·y_{max} \quad \quad \quad  \quad \quad \quad  \text{Constraint1: Linear transformatioin for y}\\
\text{(7)} \quad   &(1-u)·y_{min}  \leq  r  \leq  (1-u)·y_{max} \quad \quad \quad \quad \text{Constraint2: Linear transformatioin for y}\\
\text{(8)} \quad   &(1-u)·z_{min}  \leq  (z_{i,j}-r)  \leq  (1-u)·z_{max} \quad \quad \text{Constraint1: Linear transformatioin for z}\\
\text{(9)} \quad   &u·z_{min}  \leq  r  \leq  u·z_{max} \quad \quad \quad \quad \quad \quad \quad \quad \quad  \text{Constraint2: Linear transformatioin for z}\\
\end{align*}

Con las restricciones (6) a (9) obtenemos:
<li>Para $u = 0 \to y=0, \quad z = z \quad r = 0$</li>
<li>Para $u = 1 \to y=y, \quad z = 0 \quad r = 0$</li>


In [8]:
%%writefile paid_marketing_binary.py

from __future__ import division 
from pyomo.environ import *

model = AbstractModel()

# create indexes
model.n_channel_FB = Param(within=NonNegativeIntegers) # number of Facebook channels.
model.n_channel_GG = Param(within=NonNegativeIntegers) # number of Google channels.
model.n_day = Param(within=NonNegativeIntegers) # number of days. In this case, weekdays.

model.Channel_FB = RangeSet(1,model.n_channel_FB) # Facebook channels.
model.Channel_GG = RangeSet(1,model.n_channel_GG) # Facebook channels.
model.Day = RangeSet(1,model.n_day) # weekdays.


# create parametres
model.CPC_FB = Param(model.Channel_FB, within=NonNegativeReals) # CPC per Facebook channel.
model.CR_FB = Param(model.Channel_FB, within=NonNegativeReals) # CR per Facebook channel.
model.daily_variation_FB = Param(model.Channel_FB, model.Day, within=NonNegativeReals) # variation per weekday and channel for Facebook channels
model.CPC_GG = Param(model.Channel_GG, within=NonNegativeReals) # CPC per Facebook channel.
model.CR_GG = Param(model.Channel_GG, within=NonNegativeReals) # CR per Facebook channel.
model.daily_variation_GG = Param(model.Channel_GG, model.Day, within=NonNegativeReals) # variation per weekday and channel for Facebook channels
model.B = Param(within=NonNegativeReals) # max. budget
model.AOV = Param(within=NonNegativeReals) # average order value

model.K1 = Param(within=NonNegativeReals) # K1, percentage of budget. Minimum budget percentage for each channel and day (PERCENTAGE_MIN).
model.K2 = Param(within=NonNegativeReals) # K2, percentage of budget. Maximum budget percentage to be complied daily (PERCENTAGE_EVERYDAY_MAX).
model.K3 = Param(within=NonNegativeReals) # K3, percentage of budget. Maximum budget percentage to be complied for all channel (PERCENTAGE_ALL_CHANNEL_MAX).

# create variables
model.y = Var(model.Channel_FB, model.Day, domain=NonNegativeReals) # investment per Facebook channels.
model.z = Var(model.Channel_GG, model.Day, domain=NonNegativeReals) # investment per Google channels.
model.u = Var(domain=Binary) # binary variable.
model.r = Var(domain=Reals) # auxiliar variable for linear programming.


# create Objective Function
def objective_function_expression(model):
    return sum(sum(model.CR_FB[i]/model.CPC_FB[i]*model.daily_variation_FB[i,j]*(model.y[i,j]-model.r) 
                   for i in model.Channel_FB) for j in model.Day) * model.AOV / model.B \
            + sum(sum(model.CR_GG[i]/model.CPC_GG[i]*model.daily_variation_GG[i,j]*(model.z[i,j]+model.r) 
                   for i in model.Channel_GG) for j in model.Day) * model.AOV / model.B
model.OF = Objective(rule=objective_function_expression, sense = maximize)


# create constraints
# (1) total budget constratints
def total_budget_constraint_1_rule(model):
     return sum(sum((model.y[channel,day]-model.r) for channel in model.Channel_FB) for day in model.Day) \
    + sum(sum((model.z[channel,day]+model.r) for channel in model.Channel_GG) for day in model.Day) \
    <= model.B
model.Total_Budget_Constraint_1 = Constraint(rule = total_budget_constraint_1_rule)

# (2) Min. budget for each day constraint, K1.
def min_budget_FB_constraint_2_rule(model, channel, day):
    return model.y[channel,day]-model.r >= model.K1 * model.B * model.u
model.Min_Budget_FB_Constraint_2 = Constraint(model.Channel_FB, model.Day, rule = min_budget_FB_constraint_2_rule)

# (3) Min. budget for each day constraint, K.
def min_budget_GG_constraint_3_rule(model, channel, day):
    return model.z[channel,day]+model.r >= model.K1 * model.B * (1-model.u)
model.Min_Budget_GG_Constraint_3 = Constraint(model.Channel_GG, model.Day, rule = min_budget_GG_constraint_3_rule)

# (4) Max. budget for each day constraint, K2.
def max_budget_everyday_constraint_4_rule(model, day):
    return sum(model.y[channel,day]-model.r for channel in model.Channel_FB) \
    + sum(model.z[channel,day]+model.r for channel in model.Channel_GG) \
    <= model.K2 * model.B
model.Max_Budget_Everyday_Constraint_4 = Constraint(model.Day, rule = max_budget_everyday_constraint_4_rule)

# (5) Max. budget for any channel constraint, K3.
def max_budget_channel_constraint_5_rule(model, channel_FB, channel_GG):
    return sum(model.y[channel_FB,day]-model.r for day in model.Day) \
            + sum(model.z[channel_GG,day]+model.r for day in model.Day) \
            <= model.K3 * model.B
model.Max_Budget_Channel_Constraint_5 = Constraint(model.Channel_FB, model.Channel_GG, rule = max_budget_channel_constraint_5_rule)

# (6.1) Linear transformatioin for y (Facebook) Constraint 1 
def linear_transformation_y_constraint_61_rule(model, channel, day):
    return model.u*model.K1*model.B <= (model.y[channel, day]-model.r)
model.Linear_Transformation_y_Constraint_61 = Constraint(model.Channel_FB, model.Day, rule = linear_transformation_y_constraint_61_rule)

# (6.2) Linear transformatioin for y (Facebook) Constraint 1 
def linear_transformation_y_constraint_62_rule(model, channel, day):
    return (model.y[channel, day]-model.r) <= model.u*model.B
model.Linear_Transformation_y_Constraint_62 = Constraint(model.Channel_FB, model.Day, rule = linear_transformation_y_constraint_62_rule)

# (7.1) Linear transformatioin for y (Facebook) Constraint 2 
def linear_transformation_y_constraint_71_rule(model):
    return (1-model.u)*model.K1*model.B <= model.r 
model.Linear_Transformation_y_Constraint_71 = Constraint(rule = linear_transformation_y_constraint_71_rule)

# (7.2) Linear transformatioin for y (Facebook) Constraint 2 
def linear_transformation_y_constraint_72_rule(model):
    return model.r <= (1-model.u)*model.B
model.Linear_Transformation_y_Constraint_72 = Constraint(rule = linear_transformation_y_constraint_72_rule)

# (8.1) Linear transformatioin for z (Google) Constraint 1 
def linear_transformation_z_constraint_81_rule(model, channel, day):
    return (1-model.u)*model.K1*model.B <= (model.z[channel, day]-model.r)
model.Linear_Transformation_z_Constraint_81 = Constraint(model.Channel_GG, model.Day, rule = linear_transformation_z_constraint_81_rule)

# (8.2) Linear transformatioin for z (Google) Constraint 1 
def linear_transformation_z_constraint_82_rule(model, channel, day):
    return (model.z[channel, day]-model.r) <= (1-model.u)*model.B
model.Linear_Transformation_z_Constraint_82 = Constraint(model.Channel_GG, model.Day, rule = linear_transformation_z_constraint_82_rule)

# (9.1) Linear transformatioin forz (Google) Constraint 2 
def linear_transformation_z_constraint_91_rule(model):
    return (1-model.u)*model.K1*model.B <= model.r
model.Linear_Transformation_z_Constraint_91 = Constraint(rule = linear_transformation_z_constraint_91_rule)

# (9.2) Linear transformatioin forz (Google) Constraint 2 
def linear_transformation_z_constraint_92_rule(model):
    return model.r <= (1-model.u)*model.B
model.Linear_Transformation_z_Constraint_92 = Constraint(rule = linear_transformation_z_constraint_92_rule)

Writing paid_marketing_binary.py


In [9]:
%%writefile paid_marketing_binary.dat

param n_channel_FB := 2; # unique channels Facebook 
param n_channel_GG := 3; # unique channels Google
param n_day := 7; # number of weekdays
param B := 10000 ; # max. budget (B)
param AOV := 40; # avg. order value (AOV)
param K1 := 0.01 ; # K1, percentage of budget. Minimum budget percentage for each channel and day (PERCENTAGE_MIN).
param K2 := 0.25 ; # K2, percentage of budget. Maximum budget percentage to be complied daily (PERCENTAGE_EVERYDAY_MAX).
param K3 := 0.60 ; # K3, percentage of budget. Maximum budget percentage to be complied for all channel (PERCENTAGE_ALL_CHANNEL_MAX).

param CPC_FB:= 
1 0.25
2 0.35
;
param CR_FB:=
1 0.015
2 0.022
;
param daily_variation_FB: 1 2 3 4 5 6 7 :=
1 1.08 1.08 1.03 1.00 0.97 0.92 0.92 
2 1.03 1.02 1.00 1.00 0.95 0.88 0.86
;    

param CPC_GG :=
1	0.20
2	0.17
3	0.05
;
param CR_GG:=
1	0.010
2	0.008
3	0.001
;
param daily_variation_GG: 1 2 3 4 5 6 7 :=
1	1.10 	1.04 	1.16 	1.00 	0.96 	0.90 	0.88 
2	1.06 	1.13 	1.17 	1.00 	1.04 	0.94 	0.94 
3	1.50 	1.50 	1.25 	1.00 	1.00 	1.00 	1.00 
;




Writing paid_marketing_binary.dat


In [10]:
!pyomo solve paid_marketing_binary.py paid_marketing_binary.dat --solver=glpk --summary --solver-suffix=dual

[    0.00] Setting up Pyomo environment
[    0.00] Applying Pyomo preprocessing actions
[    0.02] Creating model
[    0.04] Applying solver
[    0.13] Processing results
    Number of solutions: 1
    Solution Information
      Gap: 0.0
      Status: optimal
      Function Value: 2.535497142857143
    Solver results file: results.yml

Solution Summary

Model unknown

  Variables:
    y : Size=14, Index=y_index
        Key    : Lower : Value  : Upper : Fixed : Stale : Domain
        (1, 1) :     0 : 2400.0 :  None : False : False : NonNegativeReals
        (1, 2) :     0 : 2400.0 :  None : False : False : NonNegativeReals
        (1, 3) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 4) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 5) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 6) :     0 :  100.0 :  None : False : False : NonNegativeReals
        (1, 7) :     0 :  100.0 :  None : False : False : NonNegativeR

In [11]:
!type results.yml

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2.53549714285714
  Upper bound: 2.53549714285714
  Number of objectives: 1
  Number of constraints: 124
  Number of variables: 38
  Number of nonzeros: 486
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.04890322685241699
# ----------------------------------------------------------
#   Solution Information
# ----------

La solución obtenida mejora el ejercicio 1b con un valor para la Función Objetivo 2,53. Es decir, mejoramos el retorno de la inversión global de las campañas de marketing. Aunque al no ser grande la mejora yo optaría por la alternativa del ejercicio 1b ya que tendríamos presencia en todos los canales.  

La solución óptima es para $u = 1$. Lo que implica que selecionamos la variable ```y``` , que son los canales de Facebook. Algo que ya preveíamos a la luz de los resultados del ejercicio 1b.  

#### Los nuevos Resultados:

  Canal             | Lunes | Martes | Miércoles | Jueves | Viernes| Sábado | Domingo |
  :----------------:|:-----:|:------:|:---------:|:------:|:------:|:------:|:-------:|
  Facebook          |  2400 |   2400 |       100 |    100 |    100 |    100 |     100 |
  Instagram Stories |   100 |    100 |      2400 |   1800 |    100 |    100 |     100 |
  
Cumpliendo las condiciones de:
<li> Presupuesto por canal mayor o igual al 1% del presupuesto total </li> 


La inversión por canal es:

  Canal             | Inversión | 
  :----------------:|:-----:|
  Facebook          | 5.300 |
  Instagram Stories | 4.700 |

Cumpliendo las condiciones de:
<li> Presupuesto máximo 10k € </li>
<li> En ningún canal se invierte más del 60% del presupuesto total </li>


Y la inversión por día es:

  |          | Lunes | Martes | Miércoles | Jueves | Viernes| Sábado | Domingo |
  |:--------:|:-----:|:------:|:---------:|:------:|:------:|:------:|:-------:|
  |Inversión | 2.500 |  2.500 |     2.500 |  1.900 |    200 |    200 |     200 |

En los que se cumple que:
<li> Presupuesto máximo 10k € </li>
<li> Ningún día supera el 20% del presupuesto total</li>


No sé porque no me sale la **sensibilidad**, solución dual.  
Igual porque la varaible binaria.  